The collaborative filter approach focuses on finding users who have given similar ratings to the same books, thus creating a link between users, to whom will be suggested books that were reviewed in a positive way. In this way, we look for associations between users, not between books.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [2]:
rating = pd.read_csv('FoodRatings2.csv', sep=';', error_bad_lines=False, encoding="latin-1")
user = pd.read_csv('UserID.csv', sep=';', error_bad_lines=False, encoding="latin-1")
food = pd.read_csv('Food.csv', sep=';', error_bad_lines=False, encoding="latin-1")
food_rating = pd.merge(rating, food, on='foodid')
cols = ['UserID', 'FoodID', 'Rating', 'Food']
##food_rating.drop(cols, axis=1, inplace=True)
food_rating

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_26616\2323556517.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  rating = pd.read_csv('FoodRatings2.csv', sep=';', error_bad_lines=False, encoding="latin-1")
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_26616\2323556517.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  user = pd.read_csv('UserID.csv', sep=';', error_bad_lines=False, encoding="latin-1")
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_26616\2323556517.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  food = pd.read_csv('Food.csv', sep=';', error_bad_lines=False, encoding="latin-1")


,userid,foodid,rating,food
0,99,9,2,Bee Hoon Goreng
1,99,9,2,Bee Hoon Goreng
2,98,9,2,Bee Hoon Goreng
3,95,9,3,Bee Hoon Goreng
4,92,9,4,Bee Hoon Goreng
...,...,...,...,...
7445,12,23,2,Curry chicken noodles
7446,11,23,5,Curry chicken noodles
7447,11,23,3,Curry chicken noodles
7448,11,23,3,Curry chicken noodles


In [3]:
rating_count = (food_rating.
     groupby(by = ['food'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'RatingCount_Food'})
     [['food', 'RatingCount_Food']])
rating_count

,food,RatingCount_Food
0,Assam Pedas,40
1,Ayam Penyet,74
2,Bak Kut Teh,80
3,Bak chor mee,76
4,Bakso,98
...,...,...
84,rawon,99
85,soup kambing,93
86,takoyaki,83
87,text,78


In [4]:
threshold = 10
rating_count = rating_count.query('RatingCount_Food >= @threshold')
rating_count.head()

,food,RatingCount_Food
0,Assam Pedas,40
1,Ayam Penyet,74
2,Bak Kut Teh,80
3,Bak chor mee,76
4,Bakso,98


In [5]:
user_rating = pd.merge(rating_count, food_rating, left_on='food', right_on='food', how='left')

In [6]:
user_rating.head()

,food,RatingCount_Food,userid,foodid,rating
0,Assam Pedas,40,99,1,4
1,Assam Pedas,40,97,1,3
2,Assam Pedas,40,84,1,4
3,Assam Pedas,40,83,1,3
4,Assam Pedas,40,83,1,2


In [7]:
user_count = (user_rating.
     groupby(by = ['userid'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'RatingCount_user'})
     [['userid', 'RatingCount_user']]
    )
user_count

,userid,RatingCount_user
0,1,33
1,2,86
2,3,69
3,4,68
4,5,69
...,...,...
96,97,79
97,98,86
98,99,89
99,100,28


In [8]:
threshold = 9
user_count = user_count.query('RatingCount_user >= @threshold')
user_count

,userid,RatingCount_user
0,1,33
1,2,86
2,3,69
3,4,68
4,5,69
...,...,...
96,97,79
97,98,86
98,99,89
99,100,28


In [9]:
combined = user_rating.merge(user_count, left_on = 'userid', right_on = 'userid', how = 'inner')

In [10]:
combined

,food,RatingCount_Food,userid,foodid,rating,RatingCount_user
0,Assam Pedas,40,99,1,4,89
1,Ayam Penyet,74,99,3,5,89
2,Bak Kut Teh,80,99,5,4,89
3,Bak Kut Teh,80,99,5,4,89
4,Bak Kut Teh,80,99,5,3,89
...,...,...,...,...,...,...
7445,takoyaki,83,1,80,2,33
7446,tom yum,79,1,85,4,33
7447,tom yum,79,1,85,4,33
7448,tom yum,79,1,85,4,33


In [14]:
combined.shape
combined.loc[combined['userid'] == 89]

,food,RatingCount_Food,userid,foodid,rating,RatingCount_user
2946,Ayam Penyet,74,89,3,3,61
2947,Bak Kut Teh,80,89,5,1,61
2948,Bakso,98,89,6,4,61
2949,Bakso,98,89,6,2,61
2950,BanMian,90,89,7,4,61
...,...,...,...,...,...,...
3002,oyster omelette,75,89,58,5,61
3003,rawon,99,89,65,2,61
3004,takoyaki,83,89,80,3,61
3005,takoyaki,83,89,80,2,61


In [77]:
print('Number of unique Food: ', combined['food'].nunique())
print('Number of unique Users: ', combined['userid'].nunique())

Number of unique Food:  89
Number of unique Users:  101


Normalize the ratings.

In [15]:
scaler = MinMaxScaler()
combined['rating'] = combined['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(combined['rating'].values.reshape(-1,1)))
combined['rating'] = rating_scaled
combined['rating']

0       0.75
1       1.00
2       0.75
3       0.75
4       0.50
        ... 
7445    0.25
7446    0.75
7447    0.75
7448    0.75
7449    0.75
Name: rating, Length: 7450, dtype: float64

Abd build the user book matrix.

In [79]:
combined = combined.drop_duplicates(['userid', 'food'])
user_food_matrix = combined.pivot(index='userid', columns='food', values='rating')
user_food_matrix.fillna(0, inplace=True)

users = user_food_matrix.index.tolist()
books = user_food_matrix.columns.tolist()

user_food_matrix = user_food_matrix.values

tf.placeholder only available in v1, so we have to work around. 

In [80]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

We will initialize the TensorFlow placeholder. Then, weights and biases are randomly initialized, the following code are taken from the book: Python Machine Learning Cook Book - Second Edition

In [81]:
num_input = combined['food'].nunique()
num_hidden_1 = 10
num_hidden_2 = 5

X = tf.placeholder(tf.float64, [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1], dtype=tf.float64)),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2], dtype=tf.float64)),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1], dtype=tf.float64)),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input], dtype=tf.float64)),
}

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2], dtype=tf.float64)),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'decoder_b2': tf.Variable(tf.random_normal([num_input], dtype=tf.float64)),
}

Now, we can build the encoder and decoder model, as follows:

In [82]:
def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    return layer_2

def decoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']), biases['decoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']), biases['decoder_b2']))
    return layer_2

We will construct the model and the predictions

In [83]:
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

y_pred = decoder_op

y_true = X

define loss function and optimizer, and minimize the squared error, and define the evaluation metrics

In [84]:
loss = tf.losses.mean_squared_error(y_true, y_pred)
optimizer = tf.train.RMSPropOptimizer(0.03).minimize(loss)
eval_x = tf.placeholder(tf.int32, )
eval_y = tf.placeholder(tf.int32, )
pre, pre_op = tf.metrics.precision(labels=eval_x, predictions=eval_y)

Initialize the variables. Because TensorFlow uses computational graphs for its operations, placeholders and variables must be initialized.

In [85]:
init = tf.global_variables_initializer()
local_init = tf.local_variables_initializer()
pred_data = pd.DataFrame()

We can finally start to train our model.

We split training data into batches, and we feed the network with them.

We train our model with vectors of user ratings, each vector represents a user and each column a book, and entries are ratings that the user gave to books. 

After a few trials, I discovered that training model for 5 epochs with a batch size of 10 would be consum enough memory. This means that the entire training set will feed our neural network 20 times, every time using 50 users.

In [86]:
with tf.Session() as session:
    epochs = 100
    batch_size = 35

    session.run(init)
    session.run(local_init)

    num_batches = int(user_food_matrix.shape[0] / batch_size)
    user_food_matrix = np.array_split(user_food_matrix, num_batches)
    
    for i in range(epochs):

        avg_cost = 0
        for batch in user_food_matrix:
            _, l = session.run([optimizer, loss], feed_dict={X: batch})
            avg_cost += l

        avg_cost /= num_batches

        print("epoch: {} Loss: {}".format(i + 1, avg_cost))

    user_food_matrix = np.concatenate(user_food_matrix, axis=0)

    preds = session.run(decoder_op, feed_dict={X: user_food_matrix})

    pred_data = pred_data.append(pd.DataFrame(preds))

    pred_data = pred_data.stack().reset_index(name='rating')
    pred_data.columns = ['userid', 'food', 'rating']
    pred_data['userid'] = pred_data['userid'].map(lambda value: users[value])
    pred_data['food'] = pred_data['food'].map(lambda value: books[value])
    
    keys = ['userid', 'food']
    index_1 = pred_data.set_index(keys).index
    index_2 = combined.set_index(keys).index

    top_ten_ranked = pred_data[~index_1.isin(index_2)]
    top_ten_ranked = top_ten_ranked.sort_values(['userid', 'rating'], ascending=[True, False])
    top_ten_ranked = top_ten_ranked.groupby('userid').head(10)

epoch: 1 Loss: 0.25147198140621185
epoch: 2 Loss: 0.2514423802495003
epoch: 3 Loss: 0.25140944868326187
epoch: 4 Loss: 0.25137293338775635
epoch: 5 Loss: 0.2513323128223419
epoch: 6 Loss: 0.25128717720508575
epoch: 7 Loss: 0.25123705714941025
epoch: 8 Loss: 0.25118137151002884
epoch: 9 Loss: 0.2511195242404938
epoch: 10 Loss: 0.25105077028274536
epoch: 11 Loss: 0.2509744018316269
epoch: 12 Loss: 0.2508895695209503
epoch: 13 Loss: 0.25079529732465744
epoch: 14 Loss: 0.25069062411785126
epoch: 15 Loss: 0.2505743205547333
epoch: 16 Loss: 0.2504451125860214
epoch: 17 Loss: 0.2503015995025635
epoch: 18 Loss: 0.2501421943306923
epoch: 19 Loss: 0.24996519088745117
epoch: 20 Loss: 0.24976854771375656
epoch: 21 Loss: 0.2495502308011055
epoch: 22 Loss: 0.24930781871080399
epoch: 23 Loss: 0.24903862178325653
epoch: 24 Loss: 0.2487398162484169
epoch: 25 Loss: 0.24840817600488663
epoch: 26 Loss: 0.24804013222455978
epoch: 27 Loss: 0.24763184040784836
epoch: 28 Loss: 0.24717902392148972
epoch: 29 Lo

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14596\655726344.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_data = pred_data.append(pd.DataFrame(preds))


In [87]:
top_ten_ranked.loc[top_ten_ranked['userid'] == 123]

,userid,food,rating
8947,123,Naan,0.474494
8945,123,Mixed Grill,0.421142
8962,123,Satay,0.409449
8949,123,Nasi lemak,0.402324
8909,123,Beef Noodle Soup,0.387966
8961,123,Roti prata,0.383087
8943,123,Mee siam,0.382068
8984,123,rawon,0.381636
8919,123,Crab bee Hoon,0.379773
8916,123,Char siu rice,0.378318


In [25]:
book_rating.loc[book_rating['User-ID'] == 278582].sort_values(by=['Book-Rating'], ascending=False)

,User-ID,ISBN,Book-Rating,Book-Title
174885,278582,0226848620,10,Chinese Bell Murders (Judge Dee Mysteries)
176582,278582,157566254X,10,"Skin Deep, Blood Red"
40008,278582,0441478123,10,The Left Hand of Darkness (Remembering Tomorrow)
174861,278582,0061044725,10,Search the Shadows
58156,278582,0451202503,10,The Songcatcher: A Ballad Novel
64570,278582,1400034779,10,The No. 1 Ladies' Detective Agency (Today Show...
175958,278582,0345350499,10,The Mists of Avalon
176314,278582,0449223558,9,Murdering Mr. Monti: A Merry Little Tale of Se...
174877,278582,0140277471,9,Blanche Cleans Up
176438,278582,0515136557,8,The Cat Who Brought Down the House
